# Réseau de neurones avec TensorFlow

In [12]:
# Charger les packages nécessaires

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
print(tf.__version__)

2.4.0


In [3]:
# Importer le jeu de données

mnist = fetch_openml(name="mnist_784", version=1)

In [4]:
# Séparer les features et la target

X, y = mnist['data'].values, mnist['target'].values


X = X.reshape(-1, 28, 28, 1)
y = y.astype(np.uint8)
X.shape

(70000, 28, 28, 1)

In [5]:
# Séparer le jeu de données en jeu d'apprentissage et de test non aléatoire avec 10000 valeurs de test

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=1/7)

In [6]:
# Normaliser le jeu de données pour un meilleur apprentissage (permet de gagner 7% d'accuracy)

X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

In [14]:
encoder = LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [18]:
y[4]

9

In [15]:
# Création du modèle avec 3 couches

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu',  input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                184330    
Total params: 203,434
Trainable params: 203,434
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Entrainement du modèle

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10


ValueError: in user code:

    C:\Users\Work\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\Work\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Work\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Work\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Work\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Work\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
       

In [10]:
#Prédiction des 30 premières valeurs et les comparer avec les valeurs réelles

y_pred = model.predict(X_test[:30])

print("Réel: ", y_test[:30])
print("Pred: ",np.argmax(y_pred, axis=1))

Réel:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1]
Pred:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1]


In [11]:
# Sauvegarder le modèle pour pouvoir le réutiliser dans l'interface graphique

model.save('mnist_model')

INFO:tensorflow:Assets written to: mnist_model\assets
